In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Direktori dataset
base_dir = "/kaggle/input/pneumonia"
train_dir = os.path.join(base_dir, "Pneumonia/train")
val_dir   = os.path.join(base_dir, "Pneumonia/val")
test_dir  = os.path.join(base_dir, "Pneumonia/test")

# Generator untuk train/val/test
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.1, zoom_range=0.1,
    horizontal_flip=True, fill_mode="nearest"
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Batch data
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=False
)
test_gen = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=False
)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model

# Load ResNet50 tanpa layer fully-connected atas
base_model = ResNet50(weights='imagenet', include_top=False,
                      input_shape=(224,224,3))
base_model.trainable = False  # beku lapisan convolutional

# Tambah lapisan kustom
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auroc')]
)


In [ ]:
# Tanpa EarlyStopping
EPOCHS = 20  # atau sesuai kebutuhan Anda, misal 30

history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    validation_data=val_gen,
    validation_steps=len(val_gen),
    epochs=EPOCHS
)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, roc_auc_score

# Prediksi probabilitas untuk data test
y_true = test_gen.classes  # 0/1 ground truth
y_prob = model.predict(test_gen, steps=len(test_gen), verbose=0).ravel()
y_pred = (y_prob >= 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)

# Hitung AUC (Area Under ROC)
auc = roc_auc_score(y_true, y_prob)
print("Test AUC:", auc)

# Akurasi
acc = np.mean(y_pred == y_true)
print(f"Test Accuracy: {acc:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Fungsi untuk memuat gambar dan menghasilkan array normalisasi [0,1]
def get_img_array(img_path, size=(224,224)):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img) / 255.0
    return np.expand_dims(array, axis=0)

# Fungsi Grad-CAM
last_conv_layer_name = "conv5_block3_out"
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_output = conv_output[0]
    heatmap = conv_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Fungsi menyimpan heatmap sebagai gambar superimposed
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)
    heatmap = np.uint8(255 * heatmap)
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    superimposed_img.save(cam_path)


In [ ]:
import glob

heatmap_dir = "/kaggle/input/data-radiolog/Pneumonia batch2 1212_1649_221124/Pneumonia batch2 1212_1649/01-10 (Sudah Blur)"
all_imgs = glob.glob(os.path.join(heatmap_dir, "*.jpg"))

pneumonia_paths, normal_paths = [], []
for img_path in all_imgs:
    img_array = get_img_array(img_path)
    pred_prob = model.predict(img_array)[0][0]
    if pred_prob >= 0.5 and len(pneumonia_paths) < 5:
        pneumonia_paths.append(img_path)
    if pred_prob < 0.5 and len(normal_paths) < 5:
        normal_paths.append(img_path)
    if len(pneumonia_paths) >= 5 and len(normal_paths) >= 5:
        break

# Buat heatmap dan simpan gambar
output_dir = "/kaggle/working/gradcam_results"
os.makedirs(output_dir, exist_ok=True)
for idx, img_path in enumerate(pneumonia_paths + normal_paths):
    img_array = get_img_array(img_path)
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    out_path = os.path.join(output_dir, f"gradcam_{idx}.jpg")
    save_and_display_gradcam(img_path, heatmap, cam_path=out_path)


In [ ]:
model.save("pneumonia_detector.keras")   # Keras SavedModel
model.save("pneumonia_detector.h5")      # HDF5
